In [86]:
import string
import re
import requests

from bs4 import BeautifulSoup
import lxml
import requests
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from collections import defaultdict
import numpy as np
import time
from IPython import display

In [6]:
url_base = 'http://buyavowel.boards.net/page/compendium'
curr_season = 36

In [7]:
puzz_list = []

In [8]:
response = requests.get(url_base.format(36))

In [9]:
response.status_code

200

In [10]:
soup = BeautifulSoup(response.text, 'lxml')

In [11]:
table = soup.find('div', {'id': 'zone_2'})

In [12]:
def get_puzzles(curr_season=35, url_base='http://buyavowel.boards.net/page/compendium'):    
    row_list = []
    for season in range(1, curr_season+1):
        url = f'{url_base}{season}'
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'lxml')
        table = soup.find('div', {'id': 'zone_2'})
        rows=[row for row in table.find_all('tr')]
        for row in rows[1:]:
            items=row.find_all('td')
            if not items:
                continue
            if items[-1].text == 'BR':
                r = 'Bonus'
            else: r= 'Round'
            row_list.append([i.text.upper() for i in items[:-1]] + [r, season])
    return pd.DataFrame(row_list, columns=['puzzle', 'category', 'date', 'round', 'season'])

In [15]:
df = get_puzzles()

In [50]:
df.date.unique()

array(['9/19/83 (#1)', '9/20/83 (#2)', '9/21/83 (#3)', ...,
       '6/6/18 (#6823)', '6/7/18 (#6824)', '6/8/18 (#6825)'], dtype=object)

In [51]:
train_dates, test_dates = train_test_split(df.date.unique(), )

In [56]:
train_puzzles = df.loc[df.date.isin(train_dates)]
test_puzzles = df.loc[df.date.isin(test_dates)]

In [60]:
train_puzzles.shape

(22095, 5)

In [69]:
test_puzzles.query('round=="Round"')

,puzzle,category,date,round,season
30,DAILY HOROSCOPE,THING,9/28/83 (#8)*,Round,1
31,ACTIONS SPEAK LOUDER THAN WORDS,PHRASE,9/28/83 (#8)*,Round,1
32,PHILADELPHIA PENNSYLVANIA,PLACE,9/28/83 (#8)*,Round,1
70,MILLION-DOLLAR SMILE,PHRASE,2/7/84 (#102)*,Round,1
71,GULF OF MEXICO,PLACE,2/7/84 (#102)*,Round,1
72,HISTORY IN THE MAKING,PHRASE,2/7/84 (#102)*,Round,1
74,OUT OF THE ORDINARY,PHRASE,2/8/84 (#103)*,Round,1
75,COOKIES CAKES AND PIES,THINGS,2/8/84 (#103)*,Round,1
76,WAYNE NEWTON,PERSON,2/8/84 (#103)*,Round,1
90,SUDDEN DEATH OVERTIME,PHRASE,4/3/84 (#137)*,Round,1


In [241]:
# unigrams = CountVectorizer(lowercase=False, analyzer='char', ngram_range=(1, 1))
ngrams = CountVectorizer(lowercase=False, analyzer='char_wb', ngram_range=(1, 5))

# X_u = unigrams.fit_transform(df.puzzle)
X_n = ngrams.fit_transform(df.puzzle)

# unigram_voc = unigrams.vocabulary_
ngram_voc = ngrams.vocabulary_

# unigrams_count = X_u.toarray().sum(axis=0)
ngrams_count = X_n.toarray().sum(axis=0)

In [242]:
def calc_ngram_freq(pattern, remaining, max_n=5, counts=ngrams_count, voc=ngram_voc):
    n = len(pattern)
#     print(pattern)
    locs = [i for i, letter in enumerate(pattern) if letter == '=']
    if len(locs) != 1:
        return {}
    missing = locs[0]
    if n > max_n:
#         print('long')
        n_sum = defaultdict(float)
        slices = [pattern[i:i+max_n] for i in range(n-max_n+1) if '=' in pattern[i:i+5]]
        for s in slices:
            slice_freqs = calc_ngram_freq(s, remaining)
            for key, value in slice_freqs.items():
                n_sum[key] += value
        return {k: v / len(slices) for k, v in n_sum.items()}

    known = set(range(n))
    known.discard(missing)
    total = 0
    n_grams = [(k, i) for k, i in voc.items() if len(k) == n]
    if n == 1:
        indices = [(ng, i) for ng, i in n_grams if ng in remaining]
    else:
        indices = [(ng[missing], i) for ng, i in n_grams 
               if [ng[j] for j in known] == [pattern[j] for j in known]
               and ng[missing] in remaining]
    n_counts = {}
    for letter, i in indices:
        total += counts[i]
        n_counts[letter] = counts[i]
    n_freqs = {k: v / total for k, v in n_counts.items()}
    return n_freqs

In [243]:
def find_chunks(masked_str):
    padded_masked_str = ' ' + masked_str + ' '
    missing_locs = [i for i, letter in enumerate(padded_masked_str) if letter == '=']
    if not missing_locs:
        return None
    chunks = []
    
    def tokenize_part(text):
        parts = text.split(' ')
        if len(parts) > 1:
            if '=' in parts[0]:
                return parts[0] + ' '
            elif '=' in parts[-1]:
                return ' ' + parts[-1]
            else:
                return ' ' + [part for part in parts if '=' in part][0] + ' '
        else:
            return text
    
    lb = 0
    cache = 0
    for loc in missing_locs:
        if lb == cache:
            cache = loc+1
            continue
        chunks.append(tokenize_part(padded_masked_str[lb:loc]))
        lb = cache
        cache = loc+1
    chunks.append(tokenize_part(padded_masked_str[lb:]))
    return chunks

In [244]:
def make_guess(masked_text, remaining, max_n=5):
    remaining_cons = remaining - set('AEIOU')
    remaining_vowels = remaining - remaining_cons
    masked_str = ''.join(masked_text)
    chunks = find_chunks(masked_str)
    guess_cons = defaultdict(float)
    guess_vowels = defaultdict(float)
    for chunk in chunks:
        chunk_freq = calc_ngram_freq(chunk, remaining, max_n)
        for letter in remaining_cons:
            guess_cons[letter] += chunk_freq.get(letter, 0)
        for letter in remaining_vowels:
            guess_vowels[letter] += chunk_freq.get(letter, 0)
    
    if guess_cons:
        best_cons = max(guess_cons.items(), key=lambda k: k[1])
    else:
        best_cons = (None, 0)
    if guess_vowels:
        best_vowel = max(guess_vowels.items(), key=lambda k: k[1])
    else:
        best_vowel = (None, 0)
    return best_cons, best_vowel

In [246]:
def update_puzzle(masked_list, under, guess, remaining):
    right = 0
    if (guess in under) and (guess in remaining):
        locs = [i for i, letter in enumerate(under) if letter == guess]
        for pos in locs:
            masked_list[pos] = guess
        print(f"There are {len(locs)} {guess}'s in the puzzle!")
        right = len(locs)
        
    else:
        print(f"Sorry, no {guess}.")
    remaining.discard(guess)
    return masked_list, right

In [247]:
def play_puzzle(text):
    guesses = 0
    masked_text = list(re.sub(r'[A-Z]', '=', text))
    remaining = set(string.ascii_uppercase)
    print(f"Puzzle: {''.join(masked_text)}")
    cons = remaining - set('AEIOU')
    vowels = remaining - cons
    print(f"Remaining Consonants: {''.join(sorted(cons))}")
    print(f"Remaining Vowels: {''.join(sorted(vowels))}")
    print()
    while masked_text != list(text):
        guesses += 1
        print(f"Round {guesses}")
        guess = make_guess(masked_text, remaining)
        print(f"Guess: {guess}")
        masked_text, _ = update_puzzle(masked_text, text, guess, remaining)
        print()

In [309]:
def full_game(puzzle_df, rounds=3):
    rules = (
        f"Let's play {rounds} rounds!"
            )
    round_puzzles = puzzle_df.query('round=="Round"')
    bonus_puzzles = puzzle_df.query('round=="Bonus"')
    print(rules)
    name = input("Player name: ")
    level = int(input("Choose computer level 1-5: "))
    while level not in range(1, 6):
        level = int(input("Invalid. Choose computer level 1-5: "))
    overall_scores = {name: 0, 'Computer': 0}
    if np.random.choice([0,1]):
#         print("Computer goes first!")
        comp_first = True
    else:
#         print(f"{name} goes first!")
        comp_first = False
    for i in range(rounds):
        print(f"{name} Total: {overall_scores[name]}")
        print(f"Computer Total: {overall_scores['Computer']}")
        if (i % 2 != comp_first):
            input(f"Computer first round {i+1}. Enter to advance ")
        else:
            input(f"{name} first round {i+1}. Enter to advance ")
        time.sleep(.1)
        display.clear_output()
        sample_row = round_puzzles.sample(1)
        winner, score = play_puzzle(sample_row.puzzle.iloc[0],
                    sample_row.category.iloc[0],
                    (i % 2 != comp_first),
                    level,
                    name
                   )
        if score < 2000:
            print('Minimum 2000 for winner!')
            time.sleep(2)
            score = 2000
        overall_scores[winner] += score
    
    print(f"{name} Total: {overall_scores[name]}")
    print(f"Computer Total: {overall_scores['Computer']}")
    if overall_scores[name] > overall_scores['Computer']:
        print('You win!')
        input('Enter to advance to bonus round!')
        sample_row = bonus_puzzles.sample(1)
        winnings = bonus_round(sample_row.puzzle.iloc[0],
                    sample_row.category.iloc[0])
        try:
            winnings = int(winnings)
            print(f"Total Winnings: {winnings + overall_scores[name]}")
        except:
            print(f'f"Total Winnings: {winnings} + {overall_scores[name]}"')
    else:
        print("Too bad, computer wins.")
        take_home = min([2000, overall_scores[name]])
        print(f'You won ${take_home}')

In [308]:
np.random.choice([
        'A New Car',
        35000,
        50000,
        'A European Vacation',
        100000
    ])

'100000'

In [303]:
def bonus_round(text, category):
    display.clear_output(wait=True)
    win = False
    bonus_prize = np.random.choice([
        'A New Car',
        35000,
        50000,
        'A European Vacation',
        100000
    ])
    masked_text = list(re.sub(r'[ABCDFGHIJKMOPQUVWXYZ]', '=', text))
    print(f"Puzzle: {''.join(masked_text)}")
    print(f'Category: {category}')
    print("Given: RSTLNE")
    s = ''
    print()
    print("Guess 3 consonants and a vowel.")
    for i in range(3):
        letter = input(f"Consonant {i+1}: ").upper()
        while (len(letter) > 1) or (letter in set('AEIOU')):
            letter = input(f"Invalid input. Consonant {i+1}: ").upper()
        s += letter
    vowel = input("Vowel: ").upper()
    while (len(vowel) > 1) or (vowel not in set('AEIOU')):
        letter = input(f"Invalid input. Vowel: ").upper()
    s += vowel
    
    for guess in s:
        locs = [i for i, letter in enumerate(text) if letter == guess]
        for pos in locs:
            masked_text[pos] = guess
    
    left = 3
    sol = ''
    display.clear_output(wait=True)
    print(f"Puzzle: {''.join(masked_text)}")
    print(f'Category: {category}')
    print("Given: RSTLNE")
    print(f'Guessed Letters: {s}')
    while sol != text and left > 0:
        

        sol = input(f"Try to solve! {left} tries left: ").upper()
        left -= 1
        if sol == text:
            win = True
    display.clear_output(wait=True)
    print(f"Puzzle: {text}")
    print(f'Category: {category}')
    print("Given: RSTLNE")
    print(f'Guessed Letters: {s}')
    print(f'Your Guess: {sol}')
    print()
    if win:
        print("CONGRATULATIONS!!!")
        print(f"You won {bonus_prize}")
        return bonus_prize
    else:
        print(f"Too bad! You could have won {bonus_prize}")
        return 0

In [250]:
puzz_values = [
    '2500',
    '500',
    '600',
    '700',
    '600',
    '650',
    '500',
    '700',
    'Bankrupt',
    '600',
    '550',
    '500',
    '600',
    'Bankrupt',
    '650',
    '500',
    '700',
    'Lose A Turn',
    '800',
    '500',
    '650',
    '500',
    '900',
    'Bankrupt',
]

In [251]:
def spin(masked_text,
                text,
                category,
                remaining,
                scores,
         name, turn):
    spaces = np.random.choice(len(puzz_values)) + int(len(puzz_values) * 0.5)
    for i in range(spaces):
        display.clear_output(wait=True)
        time.sleep(.1)
        print_puzzle_info(masked_text, category, remaining, scores, name, turn)
        print(f'Value: {puzz_values[i % len(puzz_values)]}')
    space = puzz_values[i % len(puzz_values)]
    return space

In [252]:
def player_turn(masked_text,
                text,
                category,
                remaining,
                scores,
                name):
    vowel = True
    win = False
    rd = 0
    print_puzzle_info(masked_text, category, remaining, scores, name, name)
    time.sleep(.5)
    if scores[name] < 250:
        mode = input('(S)pin or Sol(V)e? ')
    else:
        mode = input('(S)pin, (B)uy a vowel, or Sol(V)e? ')
    if (mode.upper() == 'V') or (mode.upper() == 'SOLVE'):
        sol = input('Solution: ')
        if sol.upper() == text:
            masked_text = list(text)
            win = True
            print(f'Correct! {name} wins!')
        else:
            rd += 1
            print("Sorry, incorrect.")
            print("Computer's Turn!")
        time.sleep(3)    
        return masked_text, scores, win, rd
    elif (mode.upper() == 'S') or (mode.upper() == 'SPIN'):
        space = spin(masked_text,
                text,
                category,
                remaining,
                scores, name, name)
        if space == 'Bankrupt':
            rd += 1
            print('Bankrupt!')
            print("Computer's Turn!")
            time.sleep(3)
            scores[name] = 0
            return masked_text, scores, win, rd
        elif space == 'Lose A Turn':
            rd += 1
            print('Lose a Turn!')
            print("Computer's Turn!")
            time.sleep(3)
            return masked_text, scores, win, rd
        else:
            vowel = False
            val = int(space)
    time.sleep(0.1)
    masked_text, right = player_guess(masked_text, text, vowel, remaining)
    if (mode.upper() == 'S') or (mode.upper() == 'SPIN'):
        scores[name] += right*val
    else:
        scores[name] -= 250
    if masked_text == list(text):
        print(f'Puzzle: {text}')
        print(f'{name} wins!')
        win = True
    if not right:
        rd += 1
        print("Computer's Turn!")
    time.sleep(3)
    
    return masked_text, scores, win, rd

In [259]:
def computer_turn(masked_text,
                text,
                category,
                remaining,
                scores,
                name,
                  level):
    rd = 0
    win = False
    print_puzzle_info(masked_text, category, remaining, scores, name, 'Computer')
    best_cons, best_vowel = make_guess(masked_text, remaining, level)
    if (best_cons[1] > best_vowel[1]) or (scores['Computer'] < 250):
        space = spin(masked_text,
                text,
                category,
                remaining,
                scores, name, 'Computer')
        guess = best_cons[0]
        if space == 'Bankrupt':
            rd += 1
            print('Bankrupt!')
            print(f"{name}'s Turn!")
            time.sleep(3)
            scores['Computer'] = 0
            return masked_text, scores, win, rd
        elif space == 'Lose A Turn':
            rd += 1
            print('Lose a Turn!')
            print(f"{name}'s Turn!")
            time.sleep(3)
            return masked_text, scores, win, rd
        else:
            val = int(space)
            print(f"Guess: {guess}")
            masked_text, right = update_puzzle(masked_text, text, guess, remaining)
            scores['Computer'] += val*right
    else:
        scores['Computer'] -= 250
        guess = best_vowel[0]
        print(f'Buy an {guess}')
        masked_text, right = update_puzzle(masked_text, text, guess, remaining)
    
    if masked_text == list(text):
        print(f'Puzzle: {text}')
        print('Computer wins!')
        win = True
    if not right:
        rd += 1
        print(f"{name}'s Turn!")
    time.sleep(3)
    
    return masked_text, scores, win, rd

In [254]:
def play_puzzle(text, category, comp_first, level, name):
    masked_text = list(re.sub(r'[A-Z]', '=', text))
    remaining = set(string.ascii_uppercase)
    rd = 0
    scores = {name: 0, 'Computer': 0}
    while masked_text != list(text):
        display.clear_output(wait=True)
        if (rd % 2 == 0) == comp_first:
            masked_text, scores, win, inc = computer_turn(masked_text,
                                                        text,
                                                        category,
                                                        remaining,
                                                        scores,
                                                        name,
                                                        level)
            if win:
                winner = 'Computer'
        else:
            masked_text, scores, win, inc = player_turn(masked_text,
                                                        text,
                                                        category,
                                                        remaining,
                                                        scores,
                                                        name)
            if win:
                winner = name
        rd += inc
    display.clear_output()
    return winner, scores[winner]

In [255]:
def player_guess(masked_text, text, vowel, remaining):
    kind = "vowel" if vowel else "consonant"
    guess = input(f"Guess a {kind}: ").upper()
    vowels = set('AEIOU')
    while len(guess) > 1 or ((guess in vowels) != vowel):
        guess = input(f"Invalid input. Guess a {kind}: ").upper()
    return update_puzzle(masked_text, text, guess, remaining)
    

In [260]:
def print_puzzle_info(masked, category, remaining, scores, player, turn):
    print(f'{player} score: {scores[player]}')
    print(f'Computer score: {scores["Computer"]}')
    print()
    print(f"Puzzle: {''.join(masked)}")
    print(f'Category: {category}')
    cons = remaining - set('AEIOU')
    vowels = remaining - cons
    print(f"Remaining Consonants: {''.join(sorted(cons))}")
    print(f"Remaining Vowels: {''.join(sorted(vowels))}")
    print()
    print(f"Turn: {turn}")

In [310]:
full_game(test_puzzles, 1)

Puzzle: PUSSYCAT
Category: THING
Given: RSTLNE
Guessed Letters: CDMA
Your Guess: PUSSYCAT

CONGRATULATIONS!!!
You won 50000
Total Winnings: 52350
